In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --upgrade pip
# Installs the wheel compatible with CUDA 11 and cuDNN 8.2 or newer.
# Note: wheels only available on linux.
!pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

In [ ]:
!pip install flax -U
!pip install tqdm

## Jax setup for colab

In [ ]:
# https://github.com/google/flax/issues/2263#issuecomment-1173424293
import sys
if 'google.colab' in sys.modules:
  import jax.tools.colab_tpu
  jax.tools.colab_tpu.setup_tpu()

## Dataset download
https://machinelearningmastery.com/develop-a-deep-learning-caption-generation-model-in-python/

In [ ]:
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip

In [ ]:
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip

In [ ]:
!unzip Flickr8k_Dataset.zip

In [ ]:
!unzip Flickr8k_text.zip

## Setup

In [ ]:
!pip install transformers

In [ ]:
!pip install scikit-image

In [ ]:
import torch
import numpy as np
import pandas as pd
import string
import cv2
from transformers import AutoTokenizer
from skimage import io
import os
from torchvision import transforms

In [ ]:
import tensorflow_hub as hub

In [ ]:
sentence_encoder = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
message = ["I am a sentence for which I would like to get its embedding."]

In [ ]:
embed = sentence_encoder(message)

In [ ]:
print(f'embed: {embed.shape}')
print(f'type: {type(embed)}')

In [ ]:
embed_np = np.array(embed)

In [ ]:
print(f'type: {type(embed_np)}')

In [ ]:
print(f'embed: {", ".join(map(str, list(np.squeeze(embed_np)[:3])))}, ...')

## Pre-process captions

* lower letters
* punctuation

In [ ]:
captions = []
with open('Flickr8k.token.txt', 'r') as f:
    for line in f:
        line = line.strip()
        elems = line.split('\t')
        fn_id = elems[0].split('#')  # [filename, id]
        captions.append(fn_id + [elems[1].lower()])  # [[filaneme, id, caption], ...]

In [ ]:
df_caption = pd.DataFrame(captions, columns=['image_filename', 'id', 'caption'])

In [ ]:
print(f'Unique images: {len(np.unique(df_caption.image_filename.values))}')
print(f'Total captions: {len(df_caption)}')

In [ ]:
translator = str.maketrans('', '', string.punctuation)
def remove_punctuation(text):
  return text.translate(translator)

In [ ]:
text = df_caption['caption'].iloc[0]
print(f'original: {text}')
print(f'removed: {remove_punctuation(text)}')

In [ ]:
def clean_text(text):
  cleaned_text = remove_punctuation(text)
  return cleaned_text

In [ ]:
for i, caption in enumerate(df_caption.caption.values):
  cleaned_caption = clean_text(caption)
  df_caption['caption'].iloc[i] = cleaned_caption

In [ ]:
df_caption_0 = df_caption.loc[df_caption['id'].values == '0', :]

In [ ]:
image_filenames = df_caption_0.image_filename.values
captions = df_caption_0.caption.values

In [ ]:
max_length = max([len(c.split()) for c in captions])

In [ ]:
print(f'max_length: {max_length}')

In [ ]:
print(f'{image_filenames[0]}: {captions[0]}')

## DataLoader

In [ ]:
class CLIPDataset(torch.utils.data.DataLoader):
  def __init__(self, root_dir, image_files, captions, max_length, tokenizer=None, transforms=None):
    self.image_files = [os.path.join(root_dir, f) for f in image_files]
    self.captions = captions
    self.max_length = max_length
    self.tokenizer = tokenizer if tokenizer is not None else AutoTokenizer.from_pretrained("distilbert-base-uncased")
    self.transforms = transforms

  def __getitem__(self, index):
    image_file, caption = self.image_files[index], self.captions[index]
    tokens = self.tokenizer(caption, truncation=True, padding="max_length", max_length=self.max_length)
    image = io.imread(image_file)
    if self.transforms:
      image = self.transforms(image)
    return image, tokens

  def __len__(self):
    return len(self.image_files)

In [ ]:
clip_dataset = CLIPDataset(
    root_dir='Flicker8k_Dataset',
    image_files=image_filenames,
    captions=captions,
    max_length=max_length,
    transforms=transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224)),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ])
)

## Image encoder

In [ ]:
!pip install timm

In [ ]:
import timm

In [ ]:
model = timm.create_model('resnet50', pretrained=True, num_classes=0)

In [ ]:
o = model.forward(torch.randn(2, 3, 299, 299))

In [ ]:
print(f'resnet50 feature: {o.shape}')

In [ ]:
class ImageEncoder(torch.nn.Module):
  def __init__(self, model_name, pretrained=True, num_classes=0):
    self.model = timm.create_model(
        model_name=model_name, pretrained=pretrained, num_classes=num_classes
    )

  def forward(self, x):
    return self.model(x)

## Text encoder

In [ ]:
class TextEncoder(torch.nn.Module):
  def __init__(self):
    self.sentence_encoder = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

  def forward(self, x):
    embed_np = np.array(self.sentence_encoder(x))
    return embed_np

## Projection head

In [ ]:
class ProjectionHead(torch.nn.Module):
  def __init__(self, embed_dim, proj_dim, drop_ratio):
    self.embed_dim = embed_dim
    self.proj = nn.Linear(embed_dim, proj_dim)
    self.gelu = nn.GELU()
    self.fc = nn.Linear(proj_dim, proj_dim)
    self.dropout = nn.Dropout(drop_ratio)
    self.layer_norm = nn.LayerNorm(proj_dim)

  def forward(self, x):
    x_proj = self.proj(x)
    x = self.gelu(x_proj)
    x = self.fc(x)
    x = self.dropout(x)
    x = x + x_proj
    x = self.layer_norm(x)
    return x